In [1]:
# Importamos las librerias
import pandas as pd 
import numpy as np
import json
import gzip
import ast
import pyarrow as pa
import pyarrow.parquet as pq
import fastparquet as fp

In [2]:
# Importamos los archivos generados en el ETL
archivo='steam_games.parquet'
steam_games= pd.read_parquet(archivo)
archivo='user_items.parquet'
user_items= pd.read_parquet(archivo)
archivo='user_review.parquet'
user_review= pd.read_parquet(archivo)

# Feature Engineering #


 En el dataset user_reviews se incluyen reseñas de juegos hechos por distintos usuarios. Debes crear la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP con la siguiente escala: debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo. Esta nueva columna debe reemplazar la de user_reviews.review para facilitar el trabajo de los modelos de machine learning y el análisis de datos. De no ser posible este análisis por estar ausente la reseña escrita, debe tomar el valor de 1.

In [3]:
df_s_analisis= user_review

In [4]:
# Importamo TextBlob, que implica determinar polaridad 
# (positiva, negativa o neutra) de un texto como comentario o reseña
from textblob import TextBlob
# Definimos la funcion ajustando los parametros para la polaridad
def analizar_sentimiento(review):
    if review is None:
        return 1 #Neutral
    analisis = TextBlob(review)
    polaridad = analisis.sentiment.polarity
    if polaridad > 0.1:
        return 2 #'Positivo'
    elif polaridad < -0.1:
        return 0 #'Negativo'
    else:
        return 1 #'Neutral'

In [5]:
# Aplicamos la funcion, eliminamos la columna 'review'y hacemos un head
df_s_analisis ["sentiment_analisis"]= df_s_analisis["review"].astype(str).apply(analizar_sentimiento)
df_s_analisis = df_s_analisis.drop('review', axis=1)
df_s_analisis.head(5)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analisis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,"Posted November 5, 2011.",None,1250,No ratings yet,True,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,"Posted July 15, 2011.",None,22200,No ratings yet,True,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,"Posted April 21, 2011.",None,43110,No ratings yet,True,1
3,js41637,http://steamcommunity.com/id/js41637,None,"Posted June 24, 2014.",None,251610,15 of 20 people (75%) found this review helpful,True,2
4,js41637,http://steamcommunity.com/id/js41637,None,"Posted September 8, 2013.",None,227300,0 of 1 people (0%) found this review helpful,True,1


In [6]:
# transformamos la columna posted necesaria para un posterior analisis

df_s_analisis['posted']=df_s_analisis['posted'].replace({'Posted':''},regex=True)
df_s_analisis['Año'] = df_s_analisis['posted'].str.extract(r'(\d{4})')
df_s_analisis.dropna(inplace=True)
df_s_analisis['Año'] = pd.to_numeric(df_s_analisis['Año'], errors='coerce')

df_s_analisis.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analisis,Año
24,Wackky,http://steamcommunity.com/id/Wackky,1 person found this review funny,"October 21, 2012.","Last edited November 25, 2013.",550,1 of 1 people (100%) found this review helpful,True,1,2012
214,kevinator23,http://steamcommunity.com/id/kevinator23,1 person found this review funny,"July 6, 2015.",Last edited March 8.,218620,2 of 5 people (40%) found this review helpful,True,0,2015
239,jonasdbomb,http://steamcommunity.com/id/jonasdbomb,1 person found this review funny,"March 13, 2015.","Last edited March 19, 2015.",311560,6 of 10 people (60%) found this review helpful,True,1,2015
287,getf0kinn0sc0pedm9,http://steamcommunity.com/id/getf0kinn0sc0pedm9,3 people found this review funny,"May 3, 2015.","Last edited November 8, 2015.",730,3 of 4 people (75%) found this review helpful,True,2,2015
372,1337lolroflmao,http://steamcommunity.com/id/1337lolroflmao,1 person found this review funny,"December 28, 2013.","Last edited December 28, 2013.",219740,1 of 2 people (50%) found this review helpful,True,2,2013


In [7]:
df_s_analisis.info()

<class 'pandas.core.frame.DataFrame'>
Index: 839 entries, 24 to 59046
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             839 non-null    object
 1   user_url            839 non-null    object
 2   funny               839 non-null    object
 3   posted              839 non-null    object
 4   last_edited         839 non-null    object
 5   item_id             839 non-null    int64 
 6   helpful             839 non-null    object
 7   recommend           839 non-null    bool  
 8   sentiment_analisis  839 non-null    int64 
 9   Año                 839 non-null    int64 
dtypes: bool(1), int64(3), object(6)
memory usage: 66.4+ KB


# FUNCIONES PARA LA API #

def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [91]:
# Extraemos las columnas necesarias
funcion_1_games= steam_games[['genres', 'id', 'Año']]
funcion_1_games

,genres,id,Año
0,action,761140.0,2018
1,casual,761140.0,2018
2,indie,761140.0,2018
3,simulation,761140.0,2018
4,strategy,761140.0,2018
...,...,...,...
74832,racing,610660.0,2018
74833,simulation,610660.0,2018
74834,casual,658870.0,2017
74835,indie,658870.0,2017


In [92]:
# Extraemos las columnas necesarias
funcion_1_items = user_items[['item_id', 'playtime_forever']]
funcion_1_items

,item_id,playtime_forever
0,10.0,6.0
1,20.0,0.0
2,30.0,7.0
3,40.0,0.0
4,50.0,0.0
...,...,...
5110814,373330.0,0.0
5110815,388490.0,3.0
5110816,521570.0,4.0
5110817,519140.0,3.0


In [93]:
# Definimos un dataframe uniendo los creados anterior mente valiendonos de merge 
df_funcion1 = funcion_1_items.merge(funcion_1_games, left_on='item_id', right_on='id', how='left')
df_funcion1.head()

,item_id,playtime_forever,genres,id,Año
0,10.0,6.0,action,10.0,2000
1,20.0,0.0,action,20.0,1999
2,30.0,7.0,action,30.0,2003
3,40.0,0.0,action,40.0,2001
4,50.0,0.0,action,50.0,1999


In [94]:
   
#agrupo el genero por Año y horas jugadas Y  ordena los valores de mayor a menor 
df_PlayTimeGenre = df_funcion1.groupby(["genres","Año"])["playtime_forever"].sum().reset_index()
df_PlayTimeGenre= df_PlayTimeGenre.sort_values(by="playtime_forever", ascending=False)
df_PlayTimeGenre.head(5)


,genres,Año,playtime_forever
26,action,2012,1.085635e+09
162,indie,2006,4.461153e+08
264,simulation,2006,4.445364e+08
27,action,2013,4.155222e+08
25,action,2011,3.316977e+08


In [95]:
df_PlayTimeGenre

,genres,Año,playtime_forever
26,action,2012,1.085635e+09
162,indie,2006,4.461153e+08
264,simulation,2006,4.445364e+08
27,action,2013,4.155222e+08
25,action,2011,3.316977e+08
...,...,...,...
127,education,2010,9.000000e+00
246,simulation,1988,1.000000e+00
218,rpg,1991,0.000000e+00
279,software training,2010,0.000000e+00


In [96]:
def PlayTimeGenre( genero : str ):
       
    '''
    Debe devolver año con mas horas jugadas para dicho género.
    Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

    '''
    genero = genero.lower()   
    data= df_PlayTimeGenre[df_PlayTimeGenre['genres'] == genero]
   
    año_horas = data.groupby('Año')['playtime_forever'].sum().idxmax(0)
    
   
    
    return {f"Año de lanzamiento con más horas jugadas para {genero}": año_horas}




In [98]:
PlayTimeGenre('ACtION')

{'Año de lanzamiento con más horas jugadas para action': '2012'}

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [99]:
# Extraemos las columna y creamos el dataframe necesario 
funcion_2_games= steam_games[['genres', 'id']]
funcion_2_items = user_items[['user_id', 'item_id', 'playtime_forever']]
df_UserForGenre = funcion_2_items.merge(funcion_2_games, left_on='item_id', right_on='id', how='left')
funcion_2_s_analisis = df_s_analisis[['Año', 'item_id']]
df_UserForGenre= funcion_2_s_analisis.merge(df_UserForGenre, left_on="item_id", right_on='item_id', how='left')


In [100]:
df_UserForGenre

,Año,item_id,user_id,playtime_forever,genres,id
0,2012,550,76561197970982479,1474.0,action,550.0
1,2012,550,js41637,83.0,action,550.0
2,2012,550,evcentric,82.0,action,550.0
3,2012,550,Riot-Punch,330.0,action,550.0
4,2012,550,doctr,9216.0,action,550.0
...,...,...,...,...,...,...
18131368,2015,730,76561198310819422,20016.0,action,730.0
18131369,2015,730,76561198312638244,23.0,action,730.0
18131370,2015,730,76561198313357718,2317.0,action,730.0
18131371,2015,730,POMFP0MF,2987.0,action,730.0


In [101]:
# Convertimos las columnas 'Año' y 'playtime_forever' a numerico y manejamos los errores
df_UserForGenre['Año'] = pd.to_numeric(df_UserForGenre['Año'], errors='coerce')
df_UserForGenre['playtime_forever'] = pd.to_numeric(df_UserForGenre['playtime_forever'], errors='coerce')
# Eliminamos las columnas 'item_id' y 'id'
df_UserForGenre = df_UserForGenre.drop('item_id', axis=1)
df_UserForGenre = df_UserForGenre.drop('id', axis=1)

df_UserForGenre.head()


,Año,user_id,playtime_forever,genres
0,2012,76561197970982479,1474.0,action
1,2012,js41637,83.0,action
2,2012,evcentric,82.0,action
3,2012,Riot-Punch,330.0,action
4,2012,doctr,9216.0,action


In [102]:
# Hacemos un shape para ver la cantidad de registros
df_UserForGenre.shape

(18131373, 4)

In [103]:
# Eliminamos duplicados
df_UserForGenre =df_UserForGenre.drop_duplicates(keep=False)
df_UserForGenre



,Año,user_id,playtime_forever,genres
60111,2015,js41637,790.0,action
60112,2015,js41637,790.0,adventure
60113,2015,ABSOLUTLY_NOTHING,1713.0,action
60114,2015,ABSOLUTLY_NOTHING,1713.0,adventure
60115,2015,Posiblydead,466.0,action
...,...,...,...,...
18028612,2015,76561198208451091,177.0,indie
18028613,2015,76561198208451091,177.0,strategy
18028614,2015,76561198208620779,70.0,action
18028615,2015,76561198208620779,70.0,indie


In [104]:
def UserForGenre( genero : str ): 
    '''
    Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
    Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, 
    "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}
    '''
    genero = genero.lower() 
    data= df_UserForGenre[df_UserForGenre['genres'] == genero]
    usuarios_horas = data.groupby('user_id')['playtime_forever'].sum().idxmax(0)
    horas_por_año = data.groupby('Año')['playtime_forever'].sum().reset_index()
    resultado = {
        "Usuario con más horas jugadas para " + genero: usuarios_horas,
        "Horas jugadas": [{"Año": int(row['Año']), "Horas": int(row['playtime_forever']/60)} for index, row in horas_por_año.iterrows()]
    }
    
    return resultado
    

In [105]:
UserForGenre("Action")

{'Usuario con más horas jugadas para action': 'mittensgalore',
 'Horas jugadas': [{'Año': 2012, 'Horas': 2289807},
  {'Año': 2013, 'Horas': 5555022},
  {'Año': 2014, 'Horas': 6545062},
  {'Año': 2015, 'Horas': 9054151}]}

def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [21]:
funcion_3_games= steam_games[['app_name', 'id']]
funcion_3_sentiment = df_s_analisis[['user_id', 'item_id', 'recommend', 'sentiment_analisis', 'Año']]
df_UserRecommend= funcion_3_sentiment.merge(funcion_3_games, left_on='item_id', right_on='id', how='left')
# Elimina la columna 'item_id'
df_UserRecommend = df_UserRecommend.drop('item_id', axis=1)

# Elimina la columna 'id'
df_UserRecommend = df_UserRecommend.drop('id', axis=1)
df_UserRecommend.head(2)

,user_id,recommend,sentiment_analisis,Año,app_name
0,Wackky,True,1,2012,Left 4 Dead 2
1,kevinator23,True,0,2015,PAYDAY 2


In [22]:
def UsersRecommend(anio: int):
    # Filtra el DataFrame para el año dado y los criterios especificados.
    juegos_filtrados = df_UserRecommend[(df_UserRecommend['Año'] == anio) & (df_UserRecommend['recommend'] == True) & 
                                        ((df_UserRecommend['sentiment_analisis'] == 2) | (df_UserRecommend['sentiment_analisis'] == 1))]

    # Agrupa por el nombre del juego y suma las recomendaciones y sentimientos.
    ranking = juegos_filtrados.groupby('app_name').agg({'recommend': 'sum', 'sentiment_analisis': 'sum'})

    # Calcula una puntuación que combina recomendaciones y sentimientos.
    ranking.loc[:, 'puntuacion'] = ranking['recommend'] + ranking['sentiment_analisis']

    # Ordena el ranking por la puntuación en orden descendente.
    ranking_ordenado = ranking.sort_values(by='puntuacion', ascending=False)

    # Toma los primeros 3 juegos del ranking.
    top3_juegos = ranking_ordenado.head(3)

    # Crea el formato de retorno deseado.
    resultado = [{"Puesto {}: {}".format(i + 1, juego)} for i, juego in enumerate(top3_juegos.index)]

    return resultado




In [23]:
UsersRecommend(2014)

[{'Puesto 1: Team Fortress 2'},
 {'Puesto 2: Counter-Strike: Global Offensive'},
 {'Puesto 3: DayZ'}]

def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

In [24]:
def UsersNotRecommend(año):
    # Filtra el DataFrame para el año dado y los criterios especificados.
    juegos_filtrados = df_UserRecommend[(df_UserRecommend['Año'] == año) & (df_UserRecommend['recommend'] == False) & 
                                        (df_UserRecommend['sentiment_analisis'] == 0)]

    # Agrupa por el nombre del juego y suma las recomendaciones y sentimientos.
    ranking = juegos_filtrados.groupby('app_name').agg({'recommend': 'sum', 'sentiment_analisis': 'sum'})

    # Calcula una puntuación que combina recomendaciones y sentimientos.
    ranking.loc[:, 'puntuacion'] = ranking['recommend'] + ranking['sentiment_analisis']

    # Ordena el ranking por la puntuación en orden descendente.
    ranking_ordenado = ranking.sort_values(by='puntuacion', ascending=False)

    # Toma los primeros 3 juegos del ranking.
    top3_juegos = ranking_ordenado.head(3)

    # Crea el formato de retorno deseado.
    resultado = [{"Puesto {}: {}".format(i + 1, juego)} for i, juego in enumerate(top3_juegos.index)]

    return resultado

In [25]:
UsersNotRecommend(2015)

[{'Puesto 1: 8BitMMO'}, {'Puesto 2: Alien Swarm'}, {'Puesto 3: ZOMBI'}]

def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [26]:
funcion_5_games= steam_games[['Año', 'id']]
funcion_5_sentiment = df_s_analisis[['item_id', 'sentiment_analisis']]
df_sentiment_analysis= funcion_5_games.merge(funcion_5_sentiment, left_on='id', right_on='item_id', how='left')

# Eliminamos la columna 'item_id'
df_sentiment_analysis = df_sentiment_analysis.drop('item_id', axis=1)

# Eliminamos la columna 'id'
df_sentiment_analysis = df_sentiment_analysis.drop('id', axis=1)

# Eliminamos los datos nulos y faltantes
df_sentiment_analysis.dropna(inplace=True)

df_sentiment_analysis.head(5)

,Año,sentiment_analisis
88,1998,1.0
89,1998,1.0
167,2006,1.0
168,2006,0.0
169,2006,2.0


In [27]:
def sentiment_analysis(Año):
    #Se filtran los sentimientos por año y las igualo al año que se ingresa en la consulta transformandolo en string 
    sentimientos_año= df_sentiment_analysis[df_sentiment_analysis["Año"]== int(Año)]
    
    #Se inicia una variable vacia por cada sentimiento para ir contandolos 
    Negativos = 0
    Neutral = 0
    Positivos = 0
    
    #Se itera sobre las filas de reviews_por_anio y se distibuyen los datos segun la columna "sentiment_analysis"
    for i in df_sentiment_analysis["sentiment_analisis"]:
        if i == 0:
            Negativos += 1
        elif i == 1:
            Neutral += 1 
        elif i == 2:
            Positivos += 1

    count_sentiment ={"Negative": Negativos , "Neutral" : Neutral, "Positive": Positivos}
    
    return count_sentiment


In [28]:
sentiment_analysis(2015)

{'Negative': 345, 'Neutral': 759, 'Positive': 594}

# Modelo de aprendizaje automático:  #

def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

In [62]:
df_r_juego= steam_games[['id', 'app_name', 'genres']]
df_r_juego

,id,app_name,genres
0,761140.0,Lost Summoner Kitty,action
1,761140.0,Lost Summoner Kitty,casual
2,761140.0,Lost Summoner Kitty,indie
3,761140.0,Lost Summoner Kitty,simulation
4,761140.0,Lost Summoner Kitty,strategy
...,...,...,...
74832,610660.0,Russian Roads,racing
74833,610660.0,Russian Roads,simulation
74834,658870.0,EXIT 2 - Directions,casual
74835,658870.0,EXIT 2 - Directions,indie


In [63]:
df_r_juego.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74837 entries, 0 to 74836
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        74834 non-null  float64
 1   app_name  74834 non-null  object 
 2   genres    71554 non-null  object 
dtypes: float64(1), object(2)
memory usage: 1.7+ MB


In [47]:
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator

In [64]:
df_r_juego= pd.get_dummies(df_r_juego, columns=['genres'], prefix='')

df_r_juego= df_r_juego.groupby(['id','app_name']).sum().reset_index()

df_r_juego.head(4)

,id,app_name,_accounting,_action,_adventure,_animation &amp; modeling,_audio production,_casual,_design &amp; illustration,_early access,...,_photo editing,_racing,_rpg,_simulation,_software training,_sports,_strategy,_utilities,_video production,_web publishing
0,10.0,Counter-Strike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20.0,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30.0,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40.0,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
df_r_juego['id']

0             10.0
1             20.0
2             30.0
3             40.0
4             50.0
           ...    
32126    2028055.0
32127    2028056.0
32128    2028062.0
32129    2028103.0
32130    2028850.0
Name: id, Length: 32131, dtype: float64

In [66]:
df_r_juego.shape

(32131, 24)

In [67]:
similitudes = cosine_similarity(df_r_juego.iloc[:,3:])
similitudes.shape

(32131, 32131)

In [68]:
print(similitudes)

[[1.         1.         1.         ... 1.         0.70710678 1.        ]
 [1.         1.         1.         ... 1.         0.70710678 1.        ]
 [1.         1.         1.         ... 1.         0.70710678 1.        ]
 ...
 [1.         1.         1.         ... 1.         0.70710678 1.        ]
 [0.70710678 0.70710678 0.70710678 ... 0.70710678 1.         0.70710678]
 [1.         1.         1.         ... 1.         0.70710678 1.        ]]


In [69]:
# Vamos a reducirlo para render

filas = len(df_r_juego)
df_r_juego = df_r_juego.iloc[:filas //10]
df_r_juego.shape

(3213, 24)

In [38]:
def recomendacion_juego(id):
    # Encuentra el índice del juego ingresado por ID
    juego = df_r_juego.index[df_r_juego['id'] == id]
    if juego.empty:
        return "El juego con el ID especificado no existe en la base de datos."
    
    # Extrae las características del juego ingresado
    caracteristicas =df_r_juego.iloc[juego, 3:].values.reshape(1, -1)
    
    # Calcula la similitud coseno entre el juego ingresado y todos los demás juegos
    similitudes = cosine_similarity(df_r_juego.iloc[:, 3:], caracteristicas)
    
    # Obtiene los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similitudes.argsort(axis=0)[::-1][1:6]
    indices_juegos_similares = indices_juegos_similares.flatten()[1:]
    
    # Obtiene los juegos más similares en función de los índices
    similares = df_r_juego.iloc[indices_juegos_similares]['app_name']
    
    return similares


In [54]:
recomendacion_juego(279800)

'El juego con el ID especificado no existe en la base de datos.'

In [40]:
df_s_analisis.to_parquet("df_s_analisis.parquet")
df_PlayTimeGenre.to_parquet("df_PlayTimeGenre.parquet")
df_UserForGenre.to_parquet("df_UserForGenre.parquet")
df_UserRecommend.to_parquet("df_UserRecommend.parquet")
df_sentiment_analysis.to_parquet("df_sentiment_analysis.parquet")
df_r_juego.to_parquet("df_r_juego.parquet")